In [90]:
# import stuff
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv
import math
import os
import timeit

In [91]:
def fun1(*aarg):
  for asdf in aarg:
    # print( len(asdf) )
    print( asdf )

def fun2(**quack):
  for key, value in quack.items():
    # print(key)
    print(value)

# fun1('sfd', '1', 'fff', 'ggg')
# fun2(first='what', second='horn', third='jon')

In [92]:
# load CSV file
def load_csv(filename):
  dataset = []

  with open(filename, 'r') as file:
    next(file) # skip the first line ( the 'header' )
    csv_readder = csv.reader(file)
    
    for row in csv_readder:
      if not row:
        continue
      
      dataset.append(row)

  return dataset

In [93]:
# clear values in 'dictionary'
def clear_values_in_dic(dic1):
  dic1 = dic1.fromkeys(dic1, 0)
  # print('dic1.items() >>> ' , dic1.items() )

  return dic1  


# make a 'dictionary' from sentences
def convert_to_dic(dataset):
  dic1 = {}; # empty dictionary

  for row in dataset:
    # print(row[0].lower().split(" ") ) # row, 1st item, the sentence
    # print(row[1].lower().split(" ") ) # row, 2nd item, the 'answer'
    
    word_list = row[0].lower().split(" ")
    for word in word_list:
      # print(word)
      dic1[word] = 0

  return dic1

In [94]:
# convert from 'dictionary' to 'vector'
## input: [ ['I buy an apple phone', 'happy'], ['I eat the gig apple', 'happy'] ]
## 2 sentences go in, 2 vectors come out
def convert_sentences_to_vector(dic1, sentence_list):
  vector_list = []

  for idx, row in enumerate(sentence_list):
    dic1 = clear_values_in_dic(dic1)

    sentence = row[0].lower().split()
    answer = row[1].lower()
    # print(sentence)
    # print(answer)

    for word in sentence:
      # print(word)
      for key, value in dic1.items():
        # print(key)
        if (word == key):
          dic1[key] = 1
      
    # print( list( dic1.values() ) )
    vector_list.append( [list(dic1.values() ), answer ] )

  return vector_list



def predict_answer( test_row, train_dataset, k_nay ):
  my_naybers = find_naybers(test_row, train_dataset, k_nay)
  
  my_naybers_answer = [ row[-1] for row in my_naybers ] # >> [' ', 'happy', 'happy'], >> row[-1] the last 'column' in a row
  # print( my_naybers_answer )
  the_prediction = max( set(my_naybers_answer), key=my_naybers_answer.count ) # # set() math 'set' you don't have the same stuff in a 'set'
  # print( the_prediction )

  return the_prediction



def find_naybers(test_row, train_data, k_nay):
  distance_list = []

  for train_row in train_data:
    # eu_distance = calculate_eu_distance(test_row, train_row)
    man_distance = calculate_manhat_distance(test_row, train_row)
    # print(f"train_row: {train_row}")
    # print(f"train_row[1]: {train_row[1]}")
    distance_list.append( (man_distance, train_row[1]) )

  # print( distance_list )
  distance_list.sort( key=lambda asdf: asdf[0] ) # asdf[0] is just the 'first item (man distance)' in the 'distance_list'
  # print( distance_list )

  naybers = []
  for i in range(k_nay):
    naybers.append( distance_list[i] )

  return naybers


def calculate_manhat_distance(test_row, train_row):
  distance = 0.0

  # print(f"test_row: {test_row}  train_row: {train_row}")  
  if ( len(test_row[0]) == len(train_row[0]) ):

      for a, b in zip( test_row[0], train_row[0]  ):
        # print('calculate_manhat_distance a: ', a)
        # print('calculate_manhat_distance b: ', b)

        distance = distance + abs(a-b)
  else:
    print('ERROR, vector length do not match')
    
  return distance


# v1 = [[2,3],'yes']
# v2 = [[5,7],'no']
# dis1 = calculate_manhat_distance(v1, v2)


def calculate_eu_distance(test_row, train_row):
  distance = 0.0

  # print(f"test_row: {test_row}  train_row: {train_row}")  

  if ( len(test_row[0]) == len(train_row[0]) ):

    for a, b in zip( test_row[0], train_row[0] ):
      # print('calculate_eu_distance a: ', a)
      # print('calculate_eu_distance b: ', b)

      distance = distance + (a - b)**2
  else:
    print('ERROR, vector length do not match')
    # pass

  return math.sqrt( distance )



def accuracy_met(actual, predicted):
  correct = 0

  # print(f"actual {actual}")
  # print(f"predicted {predicted}")

  for idx in range( len(actual) ): # >> idx 0,1,2
    if actual[idx] == predicted[idx]:
      correct = correct + 1
  
  # print(f"correct: {correct}  out of total: { len(actual) } "  )
  # print( correct / float( len(actual) ) * 100 )
  return correct / float( len(actual) ) * 100
    


def k_eval(test_set, train_set, actual): 
  k_evaluation = []

  for k in range(1, 22, 3):
    preds = []

    for row in test_set:
      predictors_only = row[ : -1]
      prediction = predict_answer(predictors_only, train_set, k)

      preds.append( prediction ) 
    
    curr_accuracy = accuracy_met(actual, preds)
    k_evaluation.append( (k, curr_accuracy) )

  return k_evaluation



In [103]:
##
## program start here
##

# file_name = '/content/drive/MyDrive/homework1 spring 2021/data/train_set_simple.csv'
# file_name = '/content/drive/MyDrive/homework1 spring 2021/data/train_set.csv'
file_name = '/content/drive/MyDrive/homework1 spring 2021/data/validation_set.csv'

dataset1 = load_csv(file_name)
# print(dataset1)

dic1 = convert_to_dic(dataset1)
train_set = convert_sentences_to_vector(dic1, dataset1)

actual = np.array(dataset1)[ : , -1]
# actual = np.array(dataset1)[ : 200, -1] # >> use np.array() to get the 'emotions' from each sentence

In [106]:
# train_set = train_set[ : 299]
# train_set = train_set[ 299 : ]
# print(f"train_set: {train_set} \n")


# print(f"len train_set: { len(train_set) } ")
# print(f"len actual: { len(actual) } \n")


# k_list = [3]
k_list = [3, 5, 7, 9, 11, 13, 15,17 ]


for k in k_list:
  preds3 = []

  print(f"\n len train_set: { len(train_set) } ")
  print(f"len actual: { len(actual) } \n")
  start = timeit.default_timer() # start program timer

  for row in train_set:
    test_row = row
    the_prediction = predict_answer(test_row, train_set, k)
    preds3.append( the_prediction )

  cur_accuracy = accuracy_met(actual, preds3)
  stop = timeit.default_timer() # stop program timer

  print(f"when K is {k}, accuracy %: {cur_accuracy}")
  print(f"program run time (seconds):  {stop - start} ")  



 len train_set: 311 
len actual: 311 

when K is 3, accuracy: 52.41157556270096
program run time:  15.598043903999496 

 len train_set: 311 
len actual: 311 

when K is 5, accuracy: 60.77170418006431
program run time:  15.344993569999133 

 len train_set: 311 
len actual: 311 

when K is 7, accuracy: 57.556270096463024
program run time:  15.329369007999048 

 len train_set: 311 
len actual: 311 

when K is 9, accuracy: 52.73311897106109
program run time:  15.615328634999969 

 len train_set: 311 
len actual: 311 

when K is 11, accuracy: 55.948553054662376
program run time:  15.659560326999781 

 len train_set: 311 
len actual: 311 

when K is 13, accuracy: 46.62379421221865
program run time:  15.378987602998677 

 len train_set: 311 
len actual: 311 

when K is 15, accuracy: 45.659163987138264
program run time:  15.60727263300032 

 len train_set: 311 
len actual: 311 

when K is 17, accuracy: 41.157556270096464
program run time:  15.421821160998661 


In [81]:
k_start = 3
k_stop = math.floor ( math.sqrt( len(actual) ) )
print(f"k_start: { k_start } to k_stop: {k_stop} \n")
k_eval1 = []

for k in range(k_start, k_stop, 2): # 1 to 9, increment by 2 >> 1,3,5,7,9
  preds2 = []

  for row in train_set:
    the_prediction = predict_answer(row, train_set, k) # >> 'row' is the current row you use as a 'test_row'
    preds2.append( the_prediction )
  
  current_accuracy = accuracy_met(actual, preds2)
  k_eval1.append( (k, current_accuracy) )


print(f"number of K, accuracy: {k_eval1}")

k_start: 3 to k_stop: 3 

number of K, accuracy: []


In [ ]:
###
###

In [ ]:
# test_row = [['My friend has an apple', '']]
# test_row_vector = convert_sentences_to_vector(dic1, test_row)
# print(test_row_vector)

# test_set = [ [[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], ''] ]
# test_row = [[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], ''] 

# train_row1 = [[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'happy']
# train_row2 = [[1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], 'happy']
# train_row3 = [[0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0], 'sad']

# train_set = [ [[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'happy'], 
#               [[1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], 'happy'],
#               [[0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0], 'sad'] ]
# calculate_eu_distance(test_row, train_row3)





# predict1 = predict_answer(test_row, train_dataset, 3)
# actual = ['happy', 'happy', 'sad']
# predicted = ['sad', 'happy', 'sad']